In [1]:
import cv2
import numpy as np

In [ ]:
cap = cv2.VideoCapture("./project.avi")
if cap.isOpened():
    ret,frame = cap.read()
else:
    ret =False
# background subtraction initialisation
sub = cv2.createBackgroundSubtractorMOG2()
# read video
ret, frame = cap.read()
while ret:
    ret, frame = cap.read()
    if ret:
        # image resize with no scaling
        image = cv2.resize(frame, (0, 0), None, 1.0, 1.0)
        # convert image from color to grey
        grey = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        #apply background substraction to the grey colored image
        fgmask = sub.apply(grey)
        # initialize a kernel to apply to morphological trnasformation to reduce noise
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        # Closing is reverse of Opening, Dilation followed by Erosion
        closing = cv2.morphologyEx(fgmask, cv2.MORPH_CLOSE, kernel)
        # Opening is just another name of erosion followed by dilation
        opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel)
        # increases the white region in the image or size of foreground object increases
        dilation = cv2.dilate(opening, kernel)
        # setting all pixel values above 220 to be 255 - shadow removal
        retvalbin, bins = cv2.threshold(dilation, 220, 255, cv2.THRESH_BINARY)
        _, contours, hierarchy = cv2.findContours(dilation, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        minimum_area = 400
        maximum_area = 50000
        cont_num = 0
        # goes through all contours in a single frame
        for x in range(len(contours)):
            # checks only for the parent contour
            if hierarchy[0, x, 3] == -1:
                #calculate area for each contour to place the bounding box
                contour_area = cv2.contourArea(contours[x])
                if minimum_area<contour_area<maximum_area:
                    cont_num+=1
                    cont = contours[x]
                    # compute the centre of the contour 
                    M = cv2.moments(cont)
                    cx = int(M['m10'] / M['m00'])
                    cy = int(M['m01'] / M['m00'])
                    # find coordinats of straight bounding rectangle of a contour
                    x_coord, y_coord, width, height = cv2.boundingRect(cont)
                    # draw a rectangle around the contour
                    cv2.rectangle(image, (x_coord, y_coord), (x_coord + width, y_coord + height), (0, 255, 0), 2)
                    cv2.putText(image, str(cx) + "," + str(cy), (cx + 10, cy + 10), cv2.FONT_HERSHEY_SIMPLEX,.3, (0, 0, 255), 1)
                    #cv2.putText(image, "#{}".format(x + 1), (cx - 20, cy), cv2.FONT_HERSHEY_SIMPLEX,
                    #1.0, (255, 255, 255), 2)
                    cv2.drawMarker(image, (cx, cy), (0, 255, 255), cv2.MARKER_SQUARE, markerSize=6, thickness=2,line_type=cv2.LINE_8)
                
    cv2.imshow("human detection", image)
    key = cv2.waitKey(20)
    if key == 27:
        break
cv2.destroyAllWindows()
cap.release()